In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os


In [ ]:
model_path = "/kaggle/input/tamil-kalveetu/tensorflow2/default/1/tamil_model.pth"
train_dir = "/kaggle/input/augmented/augmented/train"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [ ]:
torch.serialization.add_safe_globals([torch.nn.Sequential])

model = torch.load(
    model_path,
    map_location=device,
    weights_only=False
)
model.to(device).eval()


In [ ]:
def extract_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        _, emb = model(tensor)          # model returns (logits, embedding)
        emb = F.normalize(emb, p=2, dim=1)

    return emb.cpu()


In [ ]:
class_names = sorted(os.listdir(train_dir))
prototype_embeddings = []

for cls in class_names:
    cls_folder = os.path.join(train_dir, cls)
    img_name = os.listdir(cls_folder)[0]   # one representative image
    img_path = os.path.join(cls_folder, img_name)

    emb = extract_embedding(img_path)
    prototype_embeddings.append(emb)

prototype_matrix = torch.cat(prototype_embeddings)


In [ ]:
def embedding_predict(image_path):
    query_emb = extract_embedding(image_path)
    similarities = F.cosine_similarity(query_emb, prototype_matrix)
    idx = similarities.argmax().item()
    return class_names[idx], similarities[idx].item()


In [ ]:
test_image = "/kaggle/input/augmented/augmented/test/கௌ/aug_0_1.jpg"

pred, score = embedding_predict(test_image)
pred, score
